jan+(11*22) -------2012
2013-2024 12*12*2
2025 jan-oct   10*2
total is 331

In [32]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

def fetch_table(url , filename):
    options = webdriver.ChromeOptions()

    # Important: run in NON-HEADLESS mode (NSDL blocks headless)
    # options.add_argument("--headless")  <-- DO NOT ENABLE THIS

    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

    # Apply stealth (this is IMPORTANT)
    stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )

    driver.get(url)
    time.sleep(4)   # allow JS + table to load fully

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "lxml")
    table = soup.find("table")

    try:
        df = pd.read_html(str(table))[0]
        df.to_csv(filename, index=False)
        print("Saved table!")
    except Exception as e:
        print("❌ Error reading table from:", url)
        print("Reason:", e)
        return
    


In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import os
import urllib.parse
import random
import time

# 1) USER-AGENT ROTATION LIST
USER_AGENTS = [
    # Desktop Chrome
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",

    # Desktop Firefox
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:118.0) Gecko/20100101 Firefox/118.0",

    # Desktop Edge
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0 Safari/537.36 Edg/117.0",

    # macOS Safari
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1 Safari/605.1.15",

    # iPhone
    "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1",

    # Android Chrome
    "Mozilla/5.0 (Linux; Android 11; SM-G981B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0 Mobile Safari/537.36",
]


# 2) Function to generate a NEW session each time
def get_new_session():
    session = requests.Session()
    session.headers.update({
        "User-Agent": random.choice(USER_AGENTS),
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
    })
    return session


# 3) MAIN SCRAPER

print("Loading CDSL page...")

session = get_new_session()  # initial call with random UA
url = "https://www.cdslindia.com/Publications/ForeignPortInvestor.html"

html = session.get(url).text
soup = BeautifulSoup(html, "html.parser")

modal = soup.select_one("#FortnightlySecwiseData")
if not modal:
    raise Exception("❌ Modal not found. CDSL changed the site?")

links = modal.select("a")
print("Total links found:", len(links))

BASE = "https://www.cdslindia.com/"
records = []

for a in links:
    date_text = a.text.strip()
    raw_href = a.get("href")

    full_url = urllib.parse.urljoin(BASE, raw_href)
    full_url = full_url.replace(" ", "%20").replace(",", "%2C")

    records.append((date_text, full_url))

folder = "CDSL_Reports"
os.makedirs(folder, exist_ok=True)

summary = []


# 4) DOWNLOAD EACH REPORT
for date_text, link in records:
    print(f"\nFetching report: {date_text}")
    print("URL:", link)

    # NEW session with NEW USER-AGENT for EACH request
    session = get_new_session()

    try:
        report_html = session.get(link, timeout=10).text
        table = pd.read_html(StringIO(report_html))[0]

        table["ReportDate"] = date_text
        clean_date = date_text.replace(",", "").replace(" ", "_")
        filename = f"{folder}/CDSL_{clean_date}.csv"

        table.to_csv(filename, index=False)
        print("✔ Saved:", filename)

        summary.append({
            "date": date_text,
            "url": link,
            "file": filename
        })

    except Exception as e:
        clean_date = date_text.replace(",", "").replace(" ", "_")
        filename = f"{folder}/CDSL_{clean_date}.csv"
        fetch_table(link , filename)
        print("❌ Error processing:", link)
        print("Reason:", e)
        continue

    # random stagger delay to avoid detection
    time.sleep(random.uniform(1.2, 3.5))


# 5) SUMMARY FILE
summary_df = pd.DataFrame(summary)
summary_df.to_csv(f"{folder}/summary_links.csv", index=False)

print("\n🎉 DONE! Summary saved at:", f"{folder}/summary_links.csv")


Loading CDSL page...
Total links found: 332

Fetching report: October 31, 2025
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/October%2031%2C%202025.html
✔ Saved: CDSL_Reports/CDSL_October_31_2025.csv

Fetching report: October 15,2025
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/October%2015%2C2025.html
✔ Saved: CDSL_Reports/CDSL_October_152025.csv

Fetching report: September 30,2025
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/September%2030%2C2025.html
✔ Saved: CDSL_Reports/CDSL_September_302025.csv

Fetching report: September 15,2025
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/September%2015%2C2025.html
✔ Saved: CDSL_Reports/CDSL_September_152025.csv

Fetching report: September 15,2025
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/September%2015%2C2025.html
✔ Saved: CDSL_Reports/CDSL_September_152025.csv

Fetching report: August 31,2025
URL: https://ww

/tmp/ipykernel_4866/1099832865.py:44: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


❌ Error reading table from: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/December%2031%2C2023.htm
Reason: No tables found
❌ Error processing: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/December%2031%2C2023.htm
Reason: No tables found

Fetching report: December 15, 2023
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/December%2015%2C2023.htm
✔ Saved: CDSL_Reports/CDSL_December_15_2023.csv

Fetching report: Novemeber 30, 2023
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/November%2030%2C2023.htm
✔ Saved: CDSL_Reports/CDSL_Novemeber_30_2023.csv

Fetching report: Novemeber 15, 2023
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/November%2015%2C2023.html
✔ Saved: CDSL_Reports/CDSL_Novemeber_15_2023.csv

Fetching report: October 31, 2023
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/October%2031%2C%202023.htm
✔ Saved: CDSL_Reports/CDSL_October_31_

/tmp/ipykernel_4866/1099832865.py:44: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


❌ Error reading table from: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/July%2015%2C2023.htm
Reason: No tables found
❌ Error processing: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/July%2015%2C2023.htm
Reason: No tables found

Fetching report: June 30, 2023
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/June%2030%2C%202023.htm
✔ Saved: CDSL_Reports/CDSL_June_30_2023.csv

Fetching report: June 15, 2023
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/June%2015%2C%202023.htm
✔ Saved: CDSL_Reports/CDSL_June_15_2023.csv

Fetching report: May 31, 2023
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/May%2031%2C%202023.htm
✔ Saved: CDSL_Reports/CDSL_May_31_2023.csv

Fetching report: May 15, 2023
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/May%2015%2C%202023.htm
✔ Saved: CDSL_Reports/CDSL_May_15_2023.csv

Fetching report: April 30, 2023
URL: https://

...

Saved table!
❌ Error processing: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/April%2015%2C%202017.htm
Reason: No tables found

Fetching report: March 31, 2017
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/March%2031%2C%202017.htm


/tmp/ipykernel_4866/1099832865.py:44: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


Saved table!
❌ Error processing: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/March%2031%2C%202017.htm
Reason: No tables found

Fetching report: March 15, 2017
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/March%2015%2C%202017.htm
✔ Saved: CDSL_Reports/CDSL_March_15_2017.csv

Fetching report: February 28, 2017
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/February%2028%2C%202017.htm
✔ Saved: CDSL_Reports/CDSL_February_28_2017.csv

Fetching report: February 15, 2017
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/February%2015%2C%202017.htm
✔ Saved: CDSL_Reports/CDSL_February_15_2017.csv

Fetching report: January 31, 2017
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/January%2031%2C%202017.htm
✔ Saved: CDSL_Reports/CDSL_January_31_2017.csv

Fetching report: January 15, 2017
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/January%2015%2C%2020

/tmp/ipykernel_4866/1099832865.py:44: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


Saved table!
❌ Error processing: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/February%2015%2C%202015.htm
Reason: No tables found

Fetching report: January 31, 2015
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/January%2031%2C%202015.htm
✔ Saved: CDSL_Reports/CDSL_January_31_2015.csv

Fetching report: January 15, 2015
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/January%2015%2C%202015.htm
✔ Saved: CDSL_Reports/CDSL_January_15_2015.csv

Fetching report: December 31, 2014
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/December%2031%2C%202014.htm
✔ Saved: CDSL_Reports/CDSL_December_31_2014.csv

Fetching report: December 15, 2014
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/December%2015%2C%202014.htm
✔ Saved: CDSL_Reports/CDSL_December_15_2014.csv

Fetching report: November 30, 2014
URL: https://www.cdslindia.com/publications/FII/FortnightlySecWisePages/November30

In [35]:
import os

from pandas import array

folder = "/home/punitneh/Desktop/all_data/cds/CDSL_Reports"
map = {}
file = os.listdir(folder)
print(len(file))
for f in file:
    year = f.split("_")[0]
    if year not in map:
        map[year] = []
    map[year].append(f)

for year in map:
    print("Processing year:", year , "with files:", len(map[year]))

327
Processing year: CDSL with files: 326
Processing year: summary with files: 1
